In [50]:
import sqlite3

# Connect to the SQLite database file
conn = sqlite3.connect('Database.db')
# Replace ‘Database.db' with the path to your .db file if the file is in another directory.

# Create a cursor object
cursor = conn.cursor()

# Query to retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

# Close the connection
conn.close()


Electric_cars
Fraud_detection
Heart_disease
Insurance_Prediction
TripAdviser_Reviews
Ecommerce_data
Automobile_data
Supermarket_data


In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
# age: Age of the individual.
# gender: Gender of the individual (male/female).
# bmi: Body Mass Index (BMI) of the individual.
# children: Number of children the individual has.
# smoker: Whether the individual is a smoker (yes/no).
# region: Region where the individual resides.
# medical_history: Any specific medical history or condition the individual has.
# family_medical_history: Any family medical history or conditions.
# exercise_frequency: Frequency of exercise by the individual.
# occupation: Type of occupation of the individual.
# coverage_level: Level of health insurance coverage.
# charges: Medical charges or expenses incurred by the individual.

In [53]:
conn = sqlite3.connect('Database.db')
df = pd.read_sql_query('Select * from Insurance_Prediction' , conn)
df 

,age,gender,bmi,children,smoker,region,medical_history,family_medical_history,exercise_frequency,occupation,coverage_level,charges
0,46.0,male,21.45,5.0,yes,southeast,Diabetes,,Never,Blue collar,Premium,20460.307668871566
1,25.0,female,25.38,2.0,yes,northwest,Diabetes,High blood pressure,Occasionally,White collar,Premium,20390.8992176422
2,38.0,male,44.88,2.0,yes,southwest,,High blood pressure,Occasionally,Blue collar,Premium,20204.476301934814
3,25.0,male,19.89,0.0,no,northwest,,Diabetes,Rarely,White collar,Standard,11789.029842697417
4,49.0,male,38.21,3.0,yes,northwest,Diabetes,High blood pressure,Rarely,White collar,Standard,19268.309838159606
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,59.0,male,46.67,2.0,no,northeast,High blood pressure,,Frequently,Student,Basic,11584.134899834857
999996,33.0,male,36.83,2.0,no,northeast,,High blood pressure,Frequently,Unemployed,Basic,9834.871456313933
999997,39.0,male,39.84,0.0,yes,northeast,Heart disease,High blood pressure,Rarely,Blue collar,Standard,22076.63285624022
999998,37.0,female,45.06,4.0,yes,northeast,High blood pressure,Diabetes,Occasionally,Unemployed,Premium,20297.618727674948


In [54]:
df.sample(n=100000).to_csv("output.csv", index=False)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   age                     1000000 non-null  object
 1   gender                  1000000 non-null  object
 2   bmi                     1000000 non-null  object
 3   children                1000000 non-null  object
 4   smoker                  1000000 non-null  object
 5   region                  1000000 non-null  object
 6   medical_history         1000000 non-null  object
 7   family_medical_history  1000000 non-null  object
 8   exercise_frequency      1000000 non-null  object
 9   occupation              1000000 non-null  object
 10  coverage_level          1000000 non-null  object
 11  charges                 1000000 non-null  object
dtypes: object(12)
memory usage: 91.6+ MB


In [56]:
import pandas as pd

# Strip spaces from all string columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Ensure numeric columns are numeric
numeric_cols = ["age", "bmi", "children", "charges"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Ensure categorical columns are properly converted
category_cols = ["gender", "smoker", "region", "medical_history", "family_medical_history", "exercise_frequency", "occupation", "coverage_level"]
for col in category_cols:
    df[col] = df[col].astype("category")

# Identify and log fully missing numeric columns
empty_cols = df[numeric_cols].isnull().all()
print("Fully missing columns:", empty_cols[empty_cols].index.tolist())

# Fill missing values in numeric columns
for col in numeric_cols:
    if df[col].isnull().all():
        print(f"{col} is fully missing. Filling with 0.")
        df[col] = 0  # Fill fully missing columns with 0
    else:
        median_value = df[col].median()
        print(f"Filling {col} with median: {median_value}")
        df[col] = df[col].fillna(median_value)  # Fill partial missing with median

# Handle blanks in categorical columns
medical_cols = ["medical_history", "family_medical_history"]

# Replace blanks with 'No Disease' for medical columns
for col in medical_cols:
    print(f"Replacing blanks in '{col}' with 'No Disease'")

    # Ensure 'No Disease' is added only once
    if "No Disease" not in df[col].cat.categories:
        df[col] = df[col].cat.add_categories("No Disease")

    # Replace empty values
    df[col] = df[col].apply(lambda x: "No Disease" if x == "" else x)

# Replace blanks with the mode for other categorical columns
for col in category_cols:
    if col not in medical_cols:
        mode_value = df[col].mode()[0]
        print(f"Filling missing values in '{col}' with mode: {mode_value}")

        # Ensure mode is added only once
        if mode_value not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories(mode_value)

        # Replace empty values
        df[col] = df[col].apply(lambda x: mode_value if x == "" else x)

# Final check for remaining NaN values
print("\nRemaining missing values after cleaning:")
print(df.isnull().sum())

# Verify final data types
print("\nData types after cleaning:")
print(df.dtypes)


Fully missing columns: []
Filling age with median: 41.0
Filling bmi with median: 34.0
Filling children with median: 2.0
Filling charges with median: 16622.12797345963
Replacing blanks in 'medical_history' with 'No Disease'
Replacing blanks in 'family_medical_history' with 'No Disease'
Filling missing values in 'gender' with mode: male
Filling missing values in 'smoker' with mode: yes
Filling missing values in 'region' with mode: northeast
Filling missing values in 'exercise_frequency' with mode: Rarely
Filling missing values in 'occupation' with mode: Unemployed
Filling missing values in 'coverage_level' with mode: Basic

Remaining missing values after cleaning:
age                       0
gender                    0
bmi                       0
children                  0
smoker                    0
region                    0
medical_history           0
family_medical_history    0
exercise_frequency        0
occupation                0
coverage_level            0
charges              

In [57]:
print(df["age"].unique())  # Show first 20 unique values
print(df["children"].unique())
print(df["exercise_frequency"].unique())


[46. 25. 38. 49. 55. 64. 53. 40. 22. 21. 45. 56. 24. 36. 32. 30. 65. 35.
 60. 26. 43. 33. 44. 19. 41. 42. 61. 23. 31. 20. 62. 59. 57. 29. 34. 27.
 47. 50. 37. 52. 63. 28. 54. 51. 58. 48. 39. 18.]
[5. 2. 0. 3. 4. 1.]
['Never', 'Occasionally', 'Rarely', 'Frequently']
Categories (4, object): ['Frequently', 'Never', 'Occasionally', 'Rarely']


In [58]:
category_cols

['gender',
 'smoker',
 'region',
 'medical_history',
 'family_medical_history',
 'exercise_frequency',
 'occupation',
 'coverage_level']

In [59]:
numeric_cols

['age', 'bmi', 'children', 'charges']

In [60]:
# To find unique values in each independent variable

for col in df.columns:
  print(df[col].unique(),"\n")

[46. 25. 38. 49. 55. 64. 53. 40. 22. 21. 45. 56. 24. 36. 32. 30. 65. 35.
 60. 26. 43. 33. 44. 19. 41. 42. 61. 23. 31. 20. 62. 59. 57. 29. 34. 27.
 47. 50. 37. 52. 63. 28. 54. 51. 58. 48. 39. 18.] 

['male' 'female'] 

[21.45 25.38 44.88 ... 34.62 46.87 34.2 ] 

[5. 2. 0. 3. 4. 1.] 

['yes', 'no']
Categories (2, object): ['no', 'yes'] 

['southeast', 'northwest', 'southwest', 'northeast']
Categories (4, object): ['northeast', 'northwest', 'southeast', 'southwest'] 

['Diabetes' 'No Disease' 'High blood pressure' 'Heart disease'] 

['No Disease' 'High blood pressure' 'Diabetes' 'Heart disease'] 

['Never', 'Occasionally', 'Rarely', 'Frequently']
Categories (4, object): ['Frequently', 'Never', 'Occasionally', 'Rarely'] 

['Blue collar' 'White collar' 'Student' 'Unemployed'] 

['Premium', 'Standard', 'Basic']
Categories (3, object): ['Basic', 'Premium', 'Standard'] 

[20460.30766887 20390.89921764 20204.47630193 ... 22076.63285624
 20297.61872767 23429.72502965] 



In [61]:
print(df['age'].value_counts().head())

age
41.0    59897
43.0    20276
30.0    20217
18.0    20185
40.0    20135
Name: count, dtype: int64


In [62]:

# Function to apply log transformation
def log_transform(df, numeric_cols):
    for col in numeric_cols:
        if col == 'charges':
            df['log_charges'] = np.log(df['charges'] + 1)  # Adding 1 to avoid log(0)
    return df

# Example usage
numeric_cols = ['age', 'bmi', 'children', 'charges']
df = log_transform(df, numeric_cols)

# Verify transformation
print(df[['charges', 'log_charges']].head())


        charges  log_charges
0  20460.307669     9.926291
1  20390.899218     9.922893
2  20204.476302     9.913709
3  11789.029843     9.375010
4  19268.309838     9.866269


In [63]:
df

,age,gender,bmi,children,smoker,region,medical_history,family_medical_history,exercise_frequency,occupation,coverage_level,charges,log_charges
0,46.0,male,21.45,5.0,yes,southeast,Diabetes,No Disease,Never,Blue collar,Premium,20460.307669,9.926291
1,25.0,female,25.38,2.0,yes,northwest,Diabetes,High blood pressure,Occasionally,White collar,Premium,20390.899218,9.922893
2,38.0,male,44.88,2.0,yes,southwest,No Disease,High blood pressure,Occasionally,Blue collar,Premium,20204.476302,9.913709
3,25.0,male,19.89,0.0,no,northwest,No Disease,Diabetes,Rarely,White collar,Standard,11789.029843,9.375010
4,49.0,male,38.21,3.0,yes,northwest,Diabetes,High blood pressure,Rarely,White collar,Standard,19268.309838,9.866269
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,59.0,male,46.67,2.0,no,northeast,High blood pressure,No Disease,Frequently,Student,Basic,11584.134900,9.357478
999996,33.0,male,36.83,2.0,no,northeast,No Disease,High blood pressure,Frequently,Unemployed,Basic,9834.871456,9.193791
999997,39.0,male,39.84,0.0,yes,northeast,Heart disease,High blood pressure,Rarely,Blue collar,Standard,22076.632856,10.002320
999998,37.0,female,45.06,4.0,yes,northeast,High blood pressure,Diabetes,Occasionally,Unemployed,Premium,20297.618728,9.918308


In [81]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from sklearn.base import BaseEstimator, RegressorMixin

# Custom Wrapper for XGBoost to Convert Log Predictions to Original Scale
class LogToOriginalXGBRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, base_model=None):
        self.base_model = base_model if base_model else xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

    def fit(self, X, y):
        self.base_model.fit(X, y)  # Train on log_charges
        return self  

    def predict(self, X):
        log_preds = self.base_model.predict(X)  # Get predictions in log-scale
        return np.exp(log_preds)  # Convert to original charges


# Load Data
y_true_charges = df['charges'].copy()
df.drop(['children', 'charges'], axis=1, inplace=True, errors='ignore')

y = df["log_charges"]  # Target
X = df.drop("log_charges", axis=1) # Features

categorical_cols = ["gender", "smoker", "region", "medical_history", "family_medical_history", "exercise_frequency", "occupation", "coverage_level"]
numerical_cols = ["age", "bmi"]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, shuffle=False)


In [82]:
# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# Wrapped XGBoost Model (Auto-Conversion to Original Scale)
xgb_model_wrapped = LogToOriginalXGBRegressor()

# Final Pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb_model_wrapped)  # Wrapped model that auto-converts log to original scale
])

# **Updated Hyperparameter Grid**
param_grid = {
    'regressor__base_model__n_estimators': [500, 1000],
    'regressor__base_model__max_depth': [5, 10, 15],
    'regressor__base_model__learning_rate': [0.01, 0.1, 0.2],
    'regressor__base_model__subsample': [0.8, 0.9, 1.0],
    'regressor__base_model__colsample_bytree': [0.7, 0.8, 0.9],
    'regressor__base_model__gamma': [0, 0.1, 0.2, 0.3]
}


# **Corrected RandomizedSearchCV**
random_search = RandomizedSearchCV(
    model_pipeline, param_distributions=param_grid,
    n_iter=20, cv=3, scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=2, random_state=42
)

# Fit model with best parameters
random_search.fit(X_train, y_train)

# Save optimized model
joblib.dump(random_search.best_estimator_, "optimized_xgb_pipeline.pkl")


Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END regressor__base_model__colsample_bytree=0.8, regressor__base_model__gamma=0, regressor__base_model__learning_rate=0.01, regressor__base_model__max_depth=5, regressor__base_model__n_estimators=1000, regressor__base_model__subsample=0.9; total time=  41.3s
[CV] END regressor__base_model__colsample_bytree=0.7, regressor__base_model__gamma=0.3, regressor__base_model__learning_rate=0.1, regressor__base_model__max_depth=5, regressor__base_model__n_estimators=500, regressor__base_model__subsample=0.9; total time=  11.2s
[CV] END regressor__base_model__colsample_bytree=0.8, regressor__base_model__gamma=0, regressor__base_model__learning_rate=0.01, regressor__base_model__max_depth=15, regressor__base_model__n_estimators=1000, regressor__base_model__subsample=0.8; total time= 5.8min


/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END regressor__base_model__colsample_bytree=0.9, regressor__base_model__gamma=0.3, regressor__base_model__learning_rate=0.2, regressor__base_model__max_depth=5, regressor__base_model__n_estimators=1000, regressor__base_model__subsample=1.0; total time=  14.1s
[CV] END regressor__base_model__colsample_bytree=0.8, regressor__base_model__gamma=0.3, regressor__base_model__learning_rate=0.2, regressor__base_model__max_depth=15, regressor__base_model__n_estimators=1000, regressor__base_model__subsample=0.8; total time=  18.4s
[CV] END regressor__base_model__colsample_bytree=0.9, regressor__base_model__gamma=0, regressor__base_model__learning_rate=0.01, regressor__base_model__max_depth=15, regressor__base_model__n_estimators=1000, regressor__base_model__subsample=0.9; total time= 5.8min
[CV] END regressor__base_model__colsample_bytree=0.7, regressor__base_model__gamma=0.1, regressor__base_model__learning_rate=0.1, regressor__base_model__max_depth=10, regressor__base_model__n_estimators=1

['optimized_xgb_pipeline.pkl']

In [83]:
# Load optimized pipeline
best_pipeline = joblib.load("optimized_xgb_pipeline.pkl")

# Predictions with optimized model
y_pred_charges = best_pipeline.predict(X_test)  # Already converted to original scale

# Ensure correct y_test index
y_actual_charges = y_true_charges.loc[y_test.index]

# Optimized Model Evaluation
mse_original = mean_squared_error(y_actual_charges, y_pred_charges)
r2_original = r2_score(y_actual_charges, y_pred_charges)
mae_original = mean_absolute_error(y_actual_charges, y_pred_charges)
rmse_original = np.sqrt(mse_original)

print("\n Optimized XGBoost Model Performance:")
print(f"  - Mean Absolute Error (MAE): {mae_original:.2f}")
print(f"  - Mean Squared Error (MSE): {mse_original:.2f}")
print(f"  - Root Mean Squared Error (RMSE): {rmse_original:.2f}")
print(f"  - R^2 Score: {r2_original:.4f}")



 Optimized XGBoost Model Performance:
  - Mean Absolute Error (MAE): 424.82
  - Mean Squared Error (MSE): 293044.60
  - Root Mean Squared Error (RMSE): 541.34
  - R^2 Score: 0.9849


In [84]:
import joblib
import xgboost as xgb
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin

# Define the custom wrapper class
class LogToOriginalXGBRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, base_model=None):
        self.base_model = base_model if base_model else xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

    def fit(self, X, y):
        self.base_model.fit(X, y)
        return self

    def predict(self, X):
        log_preds = self.base_model.predict(X)
        return np.expm1(log_preds)  # Convert back to original scale

# Load the existing trained model
model_pipeline = joblib.load("optimized_xgb_pipeline.pkl")

# Re-save it properly with the correct class definition
joblib.dump(model_pipeline, "optimized_xgb_pipeline_fixed.pkl")

print("Model re-saved successfully as 'optimized_xgb_pipeline_fixed.pkl'!")


Model re-saved successfully as 'optimized_xgb_pipeline_fixed.pkl'!


In [139]:
import numpy as np
import pandas as pd
import joblib

# Load the trained pipeline
model_pipeline = joblib.load("optimized_xgb_pipeline_fixed.pkl")

# Define Raw Input Data (Unprocessed)
raw_input = pd.DataFrame({
    "age": [40],  # Numeric
    "gender": ["M"],  # Categorical
    "bmi": [26.7],  # Numeric
    "smoker": ["Yes"],  # Categorical
    "region": ["east"],  # Categorical
    "medical_history": ["none"],  # Categorical
    "family_medical_history": ["cancer"],  # Categorical
    "exercise_frequency": ["weekly"],  # Categorical
    "occupation": ["remote"],  # Categorical
    "coverage_level": ["standard"]  # Categorical
})

# Use Pipeline to Make Predictions (Original Scale Output, no need for np.exp)
predicted_charges = model_pipeline.predict(raw_input)

# Display Result
print(f" Predicted Insurance Charges: ${predicted_charges[0]:,.2f}")


 Predicted Insurance Charges: $17,169.43


In [ ]:
######--------########-------########

In [87]:
import joblib
print(joblib.__version__)


1.4.2


In [88]:
import xgboost
import joblib
import sklearn
import pandas as pd
import numpy as np

print("xgboost version:", xgboost.__version__)
print("joblib version:", joblib.__version__)
print("scikit-learn version:", sklearn.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)


xgboost version: 3.0.0
joblib version: 1.4.2
scikit-learn version: 1.4.2
pandas version: 2.2.2
numpy version: 1.26.4
